<a href="https://colab.research.google.com/github/SidharthArya/Deep_Learning_Class_Manit/blob/main/Lab/005_Multi_Headed_Human_Activity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras_multi_head

  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15612 sha256=7473f642553eaf17c953dcae7d94d63a6c6c121b975fd213a2da6a20a0a4ad8f
  Stored in directory: /root/.cache/pip/wheels/b5/b4/49/0a0c27dcb93c13af02fea254ff51d1a43a924dd4e5b7a7164d
  Created wheel for keras-self-attention: filename=keras_self_attention-0.46.0-cp36-none-any.whl size=17278 sha256=e1f90d67a995eb016b0a7d389207ba2e33fe809f9739814376a341bd23a2adf0
  Stored in directory: /root/.cache/pip/wheels/d2/2e/80/fec4c05eb23c8e13b790e26d207d6e0ffe8013fad8c6bdd4d2
Successfully built keras-multi-head keras-self-attention


In [4]:
import keras

from keras_multi_head import MultiHead
#@title Importing required packages
import sys
import os
import re
import pandas as pd
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers, Sequential, Input
import datetime
%load_ext tensorboard



In [5]:
#@title Fetching MHealth
# MHEALTH
! wget http://archive.ics.uci.edu/ml/machine-learning-databases/00319/MHEALTHDATASET.zip
! unzip MHEALTHDATASET.zip

--2021-02-11 09:48:14--  http://archive.ics.uci.edu/ml/machine-learning-databases/00319/MHEALTHDATASET.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75567983 (72M) [application/x-httpd-php]
Saving to: ‘MHEALTHDATASET.zip’

MHEALTHDATASET.zip  100%[===================>]  72.07M  17.2MB/s    in 4.2s    

2021-02-11 09:48:19 (17.2 MB/s) - ‘MHEALTHDATASET.zip’ saved [75567983/75567983]

Archive:  MHEALTHDATASET.zip
   creating: MHEALTHDATASET/
  inflating: MHEALTHDATASET/mHealth_subject1.log  
  inflating: MHEALTHDATASET/mHealth_subject10.log  
  inflating: MHEALTHDATASET/mHealth_subject2.log  
  inflating: MHEALTHDATASET/mHealth_subject3.log  
  inflating: MHEALTHDATASET/mHealth_subject4.log  
  inflating: MHEALTHDATASET/mHealth_subject5.log  
  inflating: MHEALTHDATASET/mHealth_subject6.log  
  inflating: MHEALTHDATASE

In [6]:
#@title Converting one subject to DataFrame
mHealth = pd.read_csv('MHEALTHDATASET/mHealth_subject3.log', header=None, sep='\t').loc[:, [5,6,7,8,9,10,14,15,16,17,18,19, 23]].rename(columns={5:"alX", 6:"alY", 7:"alZ", 8:"glX", 9:"glY", 10:"glZ", 14:"arX",15:"arY", 16:"arZ", 17:"grX", 18:"grY", 19:"grZ", 23:"Output"})
mHealth.head(5)

,alX,alY,alZ,glX,glY,glZ,arX,arY,arZ,grX,grY,grZ,Output
0,-0.18848,-9.8425,0.90183,0.55659,-0.83490,0.062868,-9.0897,2.0431,1.3105,-0.75490,-0.22998,0.78233,0
1,-0.36263,-9.7832,0.89263,0.55659,-0.83490,0.062868,-9.6010,1.8242,1.4245,-0.75490,-0.22998,0.78233,0
2,-0.24823,-9.8010,1.09080,0.57885,-0.84615,0.055010,-8.7446,2.4401,1.0074,-0.75490,-0.22998,0.78233,0
3,-0.25027,-9.6730,1.01380,0.57885,-0.84615,0.055010,-9.8947,1.9019,1.1888,-0.75490,-0.22998,0.78233,0
4,-0.20668,-9.9102,1.05450,0.57885,-0.84615,0.055010,-9.6522,2.0140,1.4028,-0.75294,-0.22998,0.76293,0


In [7]:
#@title Describe MHealth for all
all = mHealth[mHealth["Output"] == 14].copy()

for i in os.listdir('MHEALTHDATASET/'):
  if not re.search('README', i):
    all = all.append(pd.read_csv('MHEALTHDATASET/'+i, header=None, sep='\t').loc[:, [5,6,7,8,9,10,14,15,16,17,18,19, 23]].rename(columns={5:"alX", 6:"alY", 7:"alZ", 8:"glX", 9:"glY", 10:"glZ", 14:"arX",15:"arY", 16:"arZ", 17:"grX", 18:"grY", 19:"grZ", 23:"Output"}))


description = all.describe()

description.T

,count,mean,std,min,25%,50%,75%,max
alX,1215745.0,1.494200,3.826485,-22.1460,0.14131,1.308900,2.575800,20.0540
alY,1215745.0,-9.692878,4.171303,-19.6190,-10.20100,-9.670300,-9.042200,21.1610
alZ,1215745.0,-0.954806,5.461803,-19.3730,-2.64940,-0.016456,1.301300,25.0150
glX,1215745.0,-0.001599,0.491217,-2.1466,-0.43599,-0.014842,0.448980,60.4840
glY,1215745.0,-0.616632,0.354641,-7.7899,-0.81801,-0.707320,-0.540340,2.0113
glZ,1215745.0,-0.158781,0.546798,-2.6267,-0.59332,-0.190570,0.322200,2.7701
arX,1215745.0,-3.713413,4.763586,-22.3610,-6.07600,-2.977600,-1.193700,19.8640
arY,1215745.0,-5.805526,5.757639,-18.9720,-9.40420,-7.461500,-2.533900,22.1910
arZ,1215745.0,2.393880,3.876503,-18.2390,0.12965,1.928100,4.914700,25.7410
grX,1215745.0,-0.276106,0.527689,-8.3392,-0.70588,-0.354900,0.096078,3.3196


In [8]:
#@title Size of dataset for various subjects
for i in os.listdir('MHEALTHDATASET/'):
  if not re.search('README', i):
    print(pd.read_csv('MHEALTHDATASET/'+i, header=None, sep='\t').loc[:, [5,6,7,8,9,10,14,15,16,17,18,19, 23]].rename(columns={5:"alX", 6:"alY", 7:"alZ", 8:"glX", 9:"glY", 10:"glZ", 14:"arX",15:"arY", 16:"arZ", 17:"grX", 18:"grY", 19:"grZ", 23:"Output"}).shape)


(119808, 13)
(98304, 13)
(98304, 13)
(116736, 13)
(135168, 13)
(104448, 13)
(161280, 13)
(130561, 13)
(122112, 13)
(129024, 13)


In [9]:
#@title Importing and preprocessing dataset
SAMPLES_PER_ACTIVITY=50  #@param { "type": "integer" }

Activities = [
    "Nothing",
    "Standing",
    "Sitting and Relaxing",
    "Lying Down",
    "Walking",
    "Climbing Stairs",
    "Waist bends forward",
    "Frontal Elevation of Arms",
    "Knees Bending (Crouching)",
    "Cycling",
    "Jogging",
    "Running",
    "Jump Front & Back"
    ]


NUM_ACTIVITY=len(Activities)
X = []
Y = []

FEATURES=["alX", "alY", "alZ", "glX", "glY", "glZ", "arX", "arY", "arZ", "grX", "grY", "grZ"]
ONE_HOT_ENCODED_ACTIVITY = np.eye(NUM_ACTIVITY)
for i in os.listdir('MHEALTHDATASET/'):
  if not re.search('README', i):
    temp = pd.read_csv('MHEALTHDATASET/'+i, header=None, sep='\t').loc[:, [5,6,7,8,9,10,14,15,16,17,18,19, 23]].rename(columns={5:"alX", 6:"alY", 7:"alZ", 8:"glX", 9:"glY", 10:"glZ", 14:"arX",15:"arY", 16:"arZ", 17:"grX", 18:"grY", 19:"grZ", 23:"Output"})
    for j in range(NUM_ACTIVITY):
        output = ONE_HOT_ENCODED_ACTIVITY[j]

        if j == 0: continue
        df =np.array(temp[temp["Output"] == j].copy())
        num_recordings = int(df.shape[0]/SAMPLES_PER_ACTIVITY)
        print(f"\tThere are {num_recordings} recordings of the {Activities[j]} activity.")
        for k in range(num_recordings):
          tensor = []
          for l in range(SAMPLES_PER_ACTIVITY):
            index = k * SAMPLES_PER_ACTIVITY + l

            tensori = []
            for m,feature in enumerate(FEATURES):
              tensori.append(df[index,m] + (description[feature]["mean"]+3*description[feature]["std"]) / (3*description[feature]["std"]))
            
            tensor.append(tensori)
          X.append(tensor)
          Y.append(output)

X = np.array(X)
Y = np.array(Y)


print("Data set parsing and preparation complete.")



	There are 61 recordings of the Standing activity.
	There are 61 recordings of the Sitting and Relaxing activity.
	There are 61 recordings of the Lying Down activity.
	There are 61 recordings of the Walking activity.
	There are 61 recordings of the Climbing Stairs activity.
	There are 55 recordings of the Waist bends forward activity.
	There are 57 recordings of the Frontal Elevation of Arms activity.
	There are 54 recordings of the Knees Bending (Crouching) activity.
	There are 61 recordings of the Cycling activity.
	There are 61 recordings of the Jogging activity.
	There are 61 recordings of the Running activity.
	There are 20 recordings of the Jump Front & Back activity.
	There are 61 recordings of the Standing activity.
	There are 61 recordings of the Sitting and Relaxing activity.
	There are 61 recordings of the Lying Down activity.
	There are 61 recordings of the Walking activity.
	There are 61 recordings of the Climbing Stairs activity.
	There are 44 recordings of the Waist bend

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

In [24]:
def make_head_models(c1s, c2s, d1=0.2, p1=2):

  return Sequential(
    [

     Input(shape=input_shape),
     layers.Conv1D(filters=64, kernel_size=c1s, activation='relu'), 
     layers.Conv1D(filters=32, kernel_size=c2s, activation='relu'), 
     layers.Dropout(d1),
     layers.MaxPool1D(pool_size=p1),
     layers.Reshape((-1, 1)),
     layers.GRU(16),
     layers.Reshape((-1, 1)),
     layers.GRU(8),
     layers.Reshape((-1, 1)),
     layers.GRU(8),

     layers.Dropout(0.2),
    ]

)

In [16]:
input_shape=X.shape[1:]

In [25]:
H1=make_head_models(3, 3)

H2=make_head_models(7, 7)

H3=make_head_models(11, 11)

In [27]:
model = Sequential([
                  Input(shape=input_shape),
                  MultiHead([
                    H1, H2, H3,

                  ], name="Multiheads"),

                  layers.Dense(128, activation="relu"),

                  layers.BatchNormalization(),
                  layers.Dense(13, activation="softmax"),


])

In [36]:
#@title Compiling Model
optimizer='adam' #@param ['adam', 'sgd']
loss='mse' #@param ['categorical_crossentropy']
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

In [30]:
#@title For tensorboard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [35]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Multiheads (MultiHead)       (None, 8, 3)              63744     
_________________________________________________________________
dense_8 (Dense)              (None, 8, 128)            512       
_________________________________________________________________
batch_normalization_4 (Batch (None, 8, 128)            512       
_________________________________________________________________
dense_9 (Dense)              (None, 8, 13)             1677      
Total params: 66,445
Trainable params: 66,189
Non-trainable params: 256
_________________________________________________________________


In [38]:
#@title Training Model

epochs=50 #@param {"type": "integer" }
model.fit(x=X_train, 
          y=Y_train, 
          epochs=epochs,
          validation_split=0.2, 
          callbacks=[tensorboard_callback])

Epoch 1/50


TypeError: ignored